# <h1 align="center"><font color="red">How to Build a Simple Reasoning and Acting Agent from Scratch</font></h1>

<font color="yellow">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook está baseado no Tutorial de [kirouane Ayoub]().

Links de estudo:

* [Part 1](https://blog.gopenai.com/introduction-to-llm-agents-how-to-build-a-simple-reasoning-and-acting-agent-from-scratch-part-1-843e14686be7)

* [Part 2](https://blog.gopenai.com/building-llm-agents-from-scratch-part-2-a-conversational-search-agent-with-ollama-a4544b2291cf)

# <font color="gree">Contextualizando</font>

Esta Notebook inicia uma série sobre a construção de `agentes de IA´, começando com uma implementação básica usando `Ollama´ para executar `LLMs´ localmente. O objetivo é criar um sistema que compreenda consultas do usuário e utilize pesquisa na web para fornecer respostas informativas, demonstrando os conceitos fundamentais do design de agentes de IA.

Esta implementação serve como um ponto de partida para entender o desenvolvimento de `agentes de IA`, mas é uma versão simplificada. Em cenários reais, os agentes teriam comportamentos mais complexos, exigindo técnicas de análise sintática robustas para lidar com saídas inesperadas. Além disso, seriam necessárias estratégias abrangentes de tratamento de erros para garantir uma operação suave e confiável do sistema.

# <font color="gree">Funcionamento do Agent</font>

<img src="https://miro.medium.com/v2/resize:fit:828/format:webp/1*-c8XfC8zGlrmF8DMwWkJaA.png" alt="Minha Imagem" width="700" height="400"/>


* `Agent:` Representa o sistema geral que abrange o LLM e suas interações com o ambiente.

* `Task:` Este é o objetivo ou meta que o agente está tentando atingir. É o ponto de partida do processo.

* `LLM:` Este é o núcleo do agente, um LLM que processa informações e toma decisões.

* `Reasoning:` O LLM utiliza o raciocínio para determinar o melhor curso de ação com base na tarefa e nas informações disponíveis.

* `Tools:` O agente pode utilizar várias ferramentas (`como pesquisa na web`, `calculadoras` ou `bancos de dados`) para coletar informações adicionais ou executar ações.

* `Action:` Com base em seu raciocínio, o LLM decide uma ação a ser tomada em seu ambiente.

* `Environment:` Representa o mundo externo ou contexto no qual o agente opera. `Pode ser um site`, `um espaço físico` ou `um ambiente simulado`.

* `Result:` A ação tomada pelo agente produz um resultado no ambiente, que então realimenta a compreensão e as ações futuras do agente.


# <font color="gree">Exemplo de código</font>

## <font color="yellow">Configurando Ollama e Dependências</font>

```
curl -fsSL https://ollama.com/install.sh | sh

ollama serve
ollama pull llama3.1
```

Também:

```
pip install openai duckduckgo-search
```

## <font color="yellow">Criando um cliente OpenAI para interação LLM local</font>

In [33]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_key_openai  = os.environ['OPENAI_API_KEY']

from openai import OpenAI
client = OpenAI(api_key=Eddy_key_openai,
                base_url="http://127.0.0.1:11434/v1"
                )

## <font color="yellow">Implementando a funcionalidade de pesquisa</font>

Nosso agente precisa da capacidade de pesquisar informações na `web`. Definimos uma função `search` que utiliza a biblioteca `duckduckgo-search` para consultar `DuckDuckGo` e recuperar resultados de pesquisa relevantes.



`Esses resultados serão usados ​​para aumentar o conhecimento do LLM quando necessário.`

In [66]:
# from duckduckgo_search import DDGS

# def search(query , max_results=10):
#   results = DDGS().text(query, max_results=max_results)
#   return str("\n".join(str(results[i]) for i in range(len(results))))


from duckduckgo_search import DDGS
import logging
# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def search(query , max_results=3):
    """
    Realiza uma pesquisa web usando DuckDuckGo e retorna os resultados.
    
    Args:
        query (str): A consulta de pesquisa.
        max_results (int): O número máximo de resultados a retornar.
    
    Returns:
        str: Uma string com os resultados da pesquisa.
    """
    logging.info(f"Realizando pesquisa na Internet para: {query}")
    try:
        results = DDGS().text(query, max_results=max_results)
        logging.info("Research completed.")
        return str("\n".join(str(results[i]) for i in range(len(results))))
    except Exception as e:
        logging.error(f"Error during search: {str(e)}")
        return f"Error during search: {str(e)}"
    

In [50]:
# Testando o Duck:
from duckduckgo_search import DDGS

# Inicializa a pesquisa:
with DDGS() as ddgs:
    # Define a consulta de pesquisa:
    query = "Who won the 2024 Nobel Prize in Physics?"
    
    # Realiza a pesquisa e limita os resultados a 2:
    results = [r for r in ddgs.text(query, max_results=2)]
    
    # Exibe os resultados
    for result in results:
        print(result)

{'title': 'Press release: The Nobel Prize in Physics 2024 - NobelPrize.org', 'href': 'https://www.nobelprize.org/prizes/physics/2024/press-release/', 'body': '8 October 2024. The Royal Swedish Academy of Sciences has decided to award the Nobel Prize in Physics 2024 to. John J. Hopfield. Princeton University, NJ, USA. Geoffrey E. Hinton. University of Toronto, Canada. "for foundational discoveries and inventions that enable machine learning with artificial neural networks".'}
{'title': 'Nobel Prize in physics 2024 awarded for work on artificial intelligence ...', 'href': 'https://www.cnn.com/2024/10/08/science/nobel-prize-physics-hopfield-hinton-machine-learning-intl/index.html', 'body': 'The 2024 Nobel Prize in physics has been awarded to John Hopfield and Geoffrey Hinton for their fundamental discoveries in machine learning, which paved the way for how artificial intelligence is ...'}


## <font color="yellow">Definindo ação e extração de entrada</font>

Para orientar o comportamento do agente, precisamos entender suas ações pretendidas com base na saída do `LLM`.


Definimos uma função para extrair a `“Action”` e a `“Action Input”` da resposta do `LLM`, permitindo-nos determinar se ele pretende responder diretamente ao usuário ou realizar uma pesquisa na web.

In [67]:
import re
def extract_action_and_input(text):
  action_pattern = r"Action: (.+?)\n"
  input_pattern = r"Action Input: \"(.+?)\""
  action = re.findall(action_pattern, text)
  action_input = re.findall(input_pattern, text)
  logging.info(f"Action extracted: {action}, Action input: {action_input}")
  return action, action_input

## <font color="yellow">Elaborando o Prompt do Sistema</font>

O prompt do sistema (`system prompt`) fornece instruções ao `LLM` sobre como se comportar. Ele descreve as ferramentas disponíveis (`neste caso, pesquisa na web`) e o processo de tomada de decisão para escolher entre respostas diretas e utilizar a ferramenta de pesquisa.


`Este prompt garante que o agente siga um fluxo de trabalho lógico.`

In [68]:
System_prompt = """
Answer the following questions as best you can. Use web search only if necessary.

You have access to the following tool:

Search: useful for when you need to answer questions about current events or when you need specific information that you don't know. However, you should only use it if the user's query cannot be answered with general knowledge or a straightforward response.

You will receive a message from the human, then you should start a loop and do one of the following:

Option 1: Respond to the human directly
- If the user's question is simple, conversational, or can be answered with your existing knowledge, respond directly.
- If the user did not ask for a search or the question does not need updated or specific information, avoid using the search tool.

Use the following format when responding to the human:
Thought: Explain why a direct response is sufficient or why no search is needed.
Action: Response To Human
Action Input: "your response to the human, summarizing what you know or concluding the conversation"

Option 2: Use the search tool to answer the question.
- If you need more information to answer the question, or if the question is about a current event or specific knowledge that may not be in your training data, use the search tool.
- After receiving search results, decide whether you have enough information to answer the question or if another search is necessary.

Use the following format when using the search tool:
Thought: Explain why a search is needed or why additional searches are needed.
Action: Search
Action Input: "the input to the action, to be sent to the tool"

Remember to always decide carefully whether to search or respond directly. If you can answer the question without searching, always prefer responding directly.
"""


## <font color="yellow">Construindo o Loop de Execução do Agente</font>

O núcleo (`core`) do nosso agente está na função `run_agent`. Esta função gerencia o loop de interação entre o `usuário`, o `LLM` e a `ferramenta de busca`.


`Ele processa os prompts do usuário, os envia ao LLM, interpreta a resposta do LLM, realiza pesquisas quando necessário e, por fim, fornece uma resposta final ao usuário.`

In [69]:
def run_agent(prompt, system_prompt):
    # Prepare the initial message
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    previous_searches = set()  # Track previous search queries to avoid repetition

    while True:
        response = client.chat.completions.create(
            model="llama3.1:8b",
            messages=messages,
        )
        response_text = response.choices[0].message.content
        #print("🤗", response_text, "🤗")

        action, action_input = extract_action_and_input(response_text)
        if not action or not action_input:
            return "Failed to parse action or action input."

        if action[-1] == "Search":
            search_query = action_input[-1].strip()

            # Verifique se a consulta de pesquisa já foi realizada antes:
            if search_query in previous_searches:
                print("Repeated search detected. Stopping to prevent infinite loop.")
                break

            print(f"================ Performing search for: {search_query} ================")
            observation = search(search_query)
            print("================ Search completed !! ================")
            previous_searches.add(search_query)  # Adicionar ao conjunto de pesquisas realizadas


            messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])

        elif action[-1] == "Response To Human":
            print(f"Response: {action_input[-1]}")
            break

        # Evitar loop infinito:
        if len(previous_searches) > 3:
            print("Too many searches performed. Ending to prevent infinite loop.")
            break

    return action_input[-1]




## <font color="yellow">Exemplos ilustrativos</font>

Por fim, demonstramos as capacidades do `agente` com dois exemplos.

O primeiro mostra uma interação de conversação simples em que o agente responde diretamente a uma saudação.

In [70]:
response = run_agent(prompt="Hello!!",
          system_prompt=System_prompt)


2024-10-13 16:02:10,946 - INFO - HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-13 16:02:10,947 - INFO - Action extracted: ['Response To Human'], Action input: ['Hello! Nice to meet you!']


Response: Hello! Nice to meet you!


In [71]:
print(response)

Hello! Nice to meet you!


<font color="orange">O segundo exemplo demonstra um cenário em que o `agente` precisa realizar uma pesquisa na web para responder a uma pergunta sobre a estrutura `Ollama`, destacando sua capacidade de aproveitar `fontes externas de conhecimento`.</font>

In [76]:
response = run_agent(prompt="Tell me about the ollama framework",
          system_prompt=System_prompt)

2024-10-13 16:02:28,381 - INFO - HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-13 16:02:28,382 - INFO - Action extracted: ['Search'], Action input: ['ollama framework']
2024-10-13 16:02:28,382 - INFO - Realizando pesquisa na Internet para: ollama framework


================ Performing search for: ollama framework ================


2024-10-13 16:02:29,402 - INFO - Research completed.


================ Search completed !! ================


2024-10-13 16:02:33,327 - INFO - HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-13 16:02:33,328 - INFO - Action extracted: ['Response To Human'], Action input: ['The ollama framework is a lightweight, extensible framework for building and running language models on a local machine. It provides a simple API for creating, running, and managing models, as well as a library of pre-built models that can be easily used in various applications.']


Response: The ollama framework is a lightweight, extensible framework for building and running language models on a local machine. It provides a simple API for creating, running, and managing models, as well as a library of pre-built models that can be easily used in various applications.


In [77]:
print(response)

The ollama framework is a lightweight, extensible framework for building and running language models on a local machine. It provides a simple API for creating, running, and managing models, as well as a library of pre-built models that can be easily used in various applications.


In [74]:
response = run_agent(prompt="Who were the winners of the 2024 Nobel Prize in Physics?",
          system_prompt=System_prompt)

2024-10-13 16:02:15,759 - INFO - HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-13 16:02:15,760 - INFO - Action extracted: ['Search'], Action input: ['the winners of the 2024 Nobel Prize in Physics']
2024-10-13 16:02:15,760 - INFO - Realizando pesquisa na Internet para: the winners of the 2024 Nobel Prize in Physics


================ Performing search for: the winners of the 2024 Nobel Prize in Physics ================


2024-10-13 16:02:17,133 - INFO - Research completed.


================ Search completed !! ================


2024-10-13 16:02:19,777 - INFO - HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-13 16:02:19,778 - INFO - Action extracted: ['Respond To Human'], Action input: ['The winners of the 2024 Nobel Prize in Physics were John J. Hopfield and Geoffrey E. Hinton.']
2024-10-13 16:02:22,381 - INFO - HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-13 16:02:22,382 - INFO - Action extracted: ['Response To Human'], Action input: ['The winners of the 2024 Nobel Prize in Physics were John J. Hopfield from Princeton University, USA, and Geoffrey E. Hinton from the University of Toronto, Canada. They were awarded for their foundational discoveries and inventions that enabled machine learning with artificial neural networks.']


Response: The winners of the 2024 Nobel Prize in Physics were John J. Hopfield from Princeton University, USA, and Geoffrey E. Hinton from the University of Toronto, Canada. They were awarded for their foundational discoveries and inventions that enabled machine learning with artificial neural networks.


In [75]:
print(response)

The winners of the 2024 Nobel Prize in Physics were John J. Hopfield from Princeton University, USA, and Geoffrey E. Hinton from the University of Toronto, Canada. They were awarded for their foundational discoveries and inventions that enabled machine learning with artificial neural networks.
